# Семинар_7 Методы ансамблирования моделей. Блендинг.

Цель семинара: освоить основные техники блендинга моделей

План семинара:

* Практика - Реализуем Hard и Soft voting с помощью voting classifier и без
* Практика - заблендим модели с разными весами
* Практика - сделаем бленд вашего решения с публичным решением
* Подведение итогов - проанализируем и обсудим результаты

In [1]:
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, f1_score
from sklearn import metrics

import numpy as np
from xgboost import XGBClassifier
import xgboost as xgb
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from lightgbm import LGBMClassifier
import lightgbm as lgb

import pandas as pd
import time

pd.set_option('display.max_columns', None)

# 1. Soft Voting (15 мин).
Когда смешиваем вероятности

Сначала получим предсказания для моделей разными алгоритмами ml

Начнем с multiclass постановки задачи

In [2]:
train = pd.read_csv('/kaggle/input/seminar1/train_multiclass.csv')
test = pd.read_csv('/kaggle/input/playground-series-s4e3/test.csv')
train.head()

,id,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,Maximum_of_Luminosity,Length_of_Conveyer,TypeOfSteel_A300,TypeOfSteel_A400,Steel_Plate_Thickness,Edges_Index,Empty_Index,Square_Index,Outside_X_Index,Edges_X_Index,Edges_Y_Index,Outside_Global_Index,LogOfAreas,Log_X_Index,Log_Y_Index,Orientation_Index,Luminosity_Index,SigmoidOfAreas,Pastry,Z_Scratch,K_Scatch,Stains,Dirtiness,Bumps,Other_Faults,target
0,0,584,590,909972,909977,16,8,5,2274,113,140,1358,0,1,50,0.7393,0.4000,0.5000,0.0059,1.0000,1.0000,0.0,1.2041,0.9031,0.6990,-0.5000,-0.0104,0.1417,0,0,0,1,0,0,0,3
1,1,808,816,728350,728372,433,20,54,44478,70,111,1687,1,0,80,0.7772,0.2878,0.2581,0.0044,0.2500,1.0000,1.0,2.6365,0.7782,1.7324,0.7419,-0.2997,0.9491,0,0,0,0,0,0,1,6
2,2,39,192,2212076,2212144,11388,705,420,1311391,29,141,1400,0,1,40,0.0557,0.5282,0.9895,0.1077,0.2363,0.3857,0.0,4.0564,2.1790,2.2095,-0.0105,-0.0944,1.0000,0,0,1,0,0,0,0,2
3,3,781,789,3353146,3353173,210,16,29,3202,114,134,1387,0,1,40,0.7202,0.3333,0.3333,0.0044,0.3750,0.9310,1.0,2.3222,0.7782,1.4314,0.6667,-0.0402,0.4025,0,0,1,0,0,0,0,2
4,4,1540,1560,618457,618502,521,72,67,48231,82,111,1692,0,1,300,0.1211,0.5347,0.0842,0.0192,0.2105,0.9861,1.0,2.7694,1.4150,1.8808,0.9158,-0.2455,0.9998,0,0,0,0,0,0,1,6


In [3]:
targets = ['Pastry', 'Z_Scratch', 'K_Scatch', 'Stains', 'Dirtiness', 'Bumps', 'Other_Faults']

X = train.drop(columns=targets + ['target', 'id'])
y = train['target']

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                  random_state=43, stratify=y)

In [5]:
rf_mc = RandomForestClassifier(n_estimators=150, random_state=42)

rf_mc.fit(X_train, y_train)

RandomForestClassifier(n_estimators=150, random_state=42)

In [6]:
rf_preds = rf_mc.predict_proba(X_test)
roc_auc_lr = roc_auc_score(y_test, rf_preds, multi_class='ovo')
roc_auc_lr

0.843902041116409

In [7]:
cb_mc = CatBoostClassifier(random_seed=43,
                           eval_metric='AUC',
                           learning_rate=0.1,
                           )

cb_mc.fit(X_train, y_train, eval_set=(X_test, y_test),
          verbose=50, early_stopping_rounds=100)

0:	test: 0.7906086	best: 0.7906086 (0)	total: 113ms	remaining: 1m 52s
50:	test: 0.9128485	best: 0.9128485 (50)	total: 1.9s	remaining: 35.4s
100:	test: 0.9178482	best: 0.9178482 (100)	total: 3.63s	remaining: 32.3s
150:	test: 0.9191680	best: 0.9192460 (148)	total: 5.31s	remaining: 29.9s
200:	test: 0.9192138	best: 0.9193226 (171)	total: 6.98s	remaining: 27.7s
250:	test: 0.9194266	best: 0.9196873 (222)	total: 8.71s	remaining: 26s
300:	test: 0.9190907	best: 0.9197072 (259)	total: 10.4s	remaining: 24.2s
350:	test: 0.9188507	best: 0.9197072 (259)	total: 12.5s	remaining: 23.1s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.9197071666
bestIteration = 259

Shrink model to first 260 iterations.


In [8]:
y_pred_proba_cb = cb_mc.predict_proba(X_test)

roc_auc = roc_auc_score(y_test, y_pred_proba_cb, multi_class='ovo')
roc_auc

0.8619144795002696

In [9]:
lg_mc = LGBMClassifier(random_state=43,
        objective="multiclass",
        n_estimators=500,
        num_class=7,
        verbose=-1,
        n_jobs=-1,
)

lg_mc.fit(X_train, y_train,
        eval_set=[(X_test, y_test)],
        callbacks=[lgb.early_stopping(stopping_rounds=50)],
    )

Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[41]	valid_0's multi_logloss: 1.06599


LGBMClassifier(n_estimators=500, n_jobs=-1, num_class=7, objective='multiclass',
               random_state=43, verbose=-1)

In [10]:
y_lg_mc = lg_mc.predict_proba(X_test)
roc_auc = roc_auc_score(y_test, y_lg_mc, multi_class='ovo')
roc_auc

0.8554545880034186

* Запустите [sklearn VotingClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.VotingClassifier.html) (ссылка на доку) с нашими моделями 
* Измерьте roc_auc

In [11]:
lg_mc = LGBMClassifier(random_state=43,
        objective="multiclass",
        n_estimators=50,
        verbose=-1,
        num_class=7,
        n_jobs=-1,
)

rf_mc = RandomForestClassifier(n_estimators=150, random_state=42)

cb_mc = CatBoostClassifier(random_seed=43,
                           eval_metric='AUC',
                           iterations=260,
                           learning_rate=0.1,
                           verbose=0,
                           )

models = [('rf', rf_mc), ('lgbm', lg_mc), ('cb', cb_mc)]

voting_soft = VotingClassifier(estimators=models, voting='soft')

voting_soft.fit(X_train, y_train)

y_pred_proba_soft = voting_soft.predict_proba(X_test)

roc_auc = roc_auc_score(y_test, y_pred_proba_soft, multi_class='ovo')
roc_auc

0.860794358943277

## Блендинг с весами
* Попробуйте добавить веса нашим моделям
* Подумайте, какой модели лучше присвоить наибольший вес, а какой наименьший 

In [12]:
weights = [0.1, 0.3, 0.6]
voting_soft = VotingClassifier(estimators=models, voting='soft', weights=weights)

voting_soft.fit(X_train, y_train)

y_pred_proba_soft = voting_soft.predict_proba(X_test)
y_pred_soft = voting_soft.predict(X_test)

roc_auc = roc_auc_score(y_test, y_pred_proba_soft, multi_class='ovo')
f1_soft = f1_score(y_test, y_pred_soft, average='macro') # Подсчитаем F1
roc_auc, f1_soft

(0.8631931357484875, 0.4776063194815121)

* Скор подрос - класс

# 2. Теперь попробуем сами реализовать hard voting (10 мин)
Выбираем самую встречаемую метку из всех предсказаний

Нам нужно предиктить не вероятности, а значение класса. 

Измерим качество метрикой [f1 score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html).

In [13]:
rf_mc = RandomForestClassifier(n_estimators=150, random_state=42)

rf_mc.fit(X_train, y_train)

rf_preds = rf_mc.predict(X_test)
f1_score(y_test, rf_preds, average='macro')

0.43421859356527065

In [14]:
%%time
cb_mc = CatBoostClassifier(random_seed=43,
                           eval_metric='AUC',
                           learning_rate=0.1,
                           task_type='CPU', # по умолчанию
                           thread_count=-1, # по умолчанию
                           )

cb_mc.fit(X_train, y_train, eval_set=(X_test, y_test),
          verbose=50, early_stopping_rounds=100)

0:	test: 0.7906086	best: 0.7906086 (0)	total: 55.1ms	remaining: 55s
50:	test: 0.9128485	best: 0.9128485 (50)	total: 1.98s	remaining: 36.8s
100:	test: 0.9178482	best: 0.9178482 (100)	total: 3.8s	remaining: 33.9s
150:	test: 0.9191680	best: 0.9192460 (148)	total: 5.59s	remaining: 31.4s
200:	test: 0.9192138	best: 0.9193226 (171)	total: 7.35s	remaining: 29.2s
250:	test: 0.9194266	best: 0.9196873 (222)	total: 9.11s	remaining: 27.2s
300:	test: 0.9190907	best: 0.9197072 (259)	total: 11s	remaining: 25.5s
350:	test: 0.9188507	best: 0.9197072 (259)	total: 12.7s	remaining: 23.6s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.9197071666
bestIteration = 259

Shrink model to first 260 iterations.
CPU times: user 48 s, sys: 1.45 s, total: 49.5 s
Wall time: 13.2 s


In [15]:
y_cb_mc = cb_mc.predict(X_test).ravel()
f1_cb = f1_score(y_test, y_cb_mc, average='macro')
f1_cb

0.47698032401709123

In [16]:
%%time
lg_mc = LGBMClassifier(random_state=43,
        objective="multiclass",
        n_estimators=500,
        num_class=8,
        verbose=-1,
        n_jobs=-1,
)

lg_mc.fit(X_train, y_train,
        eval_set=[(X_test, y_test)],
        callbacks=[lgb.early_stopping(stopping_rounds=50)],
    )

Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[41]	valid_0's multi_logloss: 1.06599
CPU times: user 5.14 s, sys: 11.4 ms, total: 5.15 s
Wall time: 5.16 s


LGBMClassifier(n_estimators=500, n_jobs=-1, num_class=8, objective='multiclass',
               random_state=43, verbose=-1)

In [17]:
y_lg_mc = lg_mc.predict(X_test)
f1_lg = f1_score(y_test, y_lg_mc, average='macro')
f1_lg

0.47688686133400016

Мы получили hard labels предсказания. Они хранятся в переменных `y_cb_mc`, `y_lg_mc`, `rf_preds`.

Сделайте бленд решением большинства, например с помощью [функции `mode`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.mode.html) из `scipy` или другим способом.

In [18]:
from scipy.stats import mode

predictions = np.vstack((rf_preds, y_cb_mc, y_lg_mc))
predictions

array([[1, 6, 5, ..., 3, 6, 1],
       [6, 1, 5, ..., 3, 5, 1],
       [1, 1, 5, ..., 3, 5, 1]])

In [19]:
hard_voting_predictions, _ = mode(predictions, axis=0)

In [20]:
f1_hard = f1_score(y_test, hard_voting_predictions, average='macro')
f1_hard

0.46310167269489755

# 3. Бленд в задаче мультилейбл. (10 мин)

In [21]:
targets = ['Pastry', 'Z_Scratch', 'K_Scatch', 'Stains', 'Dirtiness', 'Bumps', 'Other_Faults']

X = train.drop(columns=targets + ['target', 'id'])
y = train[targets]

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                  random_state=43)

Приведите предсказания y_pred_proba в нужный для нас формат


In [23]:
clf_rf = RandomForestClassifier(n_estimators=80, random_state=42)

clf_rf.fit(X_train, y_train)

y_pred_proba = clf_rf.predict_proba(X_test)

y_pred_proba = [element[:, 1].reshape(-1, 1) for element in y_pred_proba]
y_pred_proba_rf = np.hstack(y_pred_proba)

roc_auc = roc_auc_score(y_test, y_pred_proba_rf, multi_class='ovo')
roc_auc

0.8696074513197496

In [24]:
clf_xb = xgb.XGBClassifier(tree_method="hist",
                        seed=42,
                        n_estimators=80,
                        early_stopping_rounds=10,
                       )
eval_set = [(X_train, y_train), (X_test, y_test)]
clf_xb.fit(X_train, y_train, eval_set=eval_set, verbose=5)

y_pred_proba_xb = clf_xb.predict_proba(X_test)
roc_auc = roc_auc_score(y_test, y_pred_proba_xb, multi_class='ovo')
roc_auc

[0]	validation_0-logloss:0.32725	validation_1-logloss:0.33221
[5]	validation_0-logloss:0.22112	validation_1-logloss:0.24060
[10]	validation_0-logloss:0.19573	validation_1-logloss:0.22735
[15]	validation_0-logloss:0.18297	validation_1-logloss:0.22570
[20]	validation_0-logloss:0.17415	validation_1-logloss:0.22625
[25]	validation_0-logloss:0.16567	validation_1-logloss:0.22759
[26]	validation_0-logloss:0.16407	validation_1-logloss:0.22792


0.8786061873598214

* Какую лосс-функцию надо передать в [Катбуст](https://catboost.ai/en/docs/concepts/loss-functions), чтобы запустилось мультилэйбл обучение?

In [25]:
clf_cb = CatBoostClassifier(random_seed=42,
                         #iterations=200,
                         early_stopping_rounds=50,
                         loss_function='MultiLogloss',
                         verbose=50)

clf_cb.fit(X_train, y_train, eval_set=(X_test, y_test))

y_pred_proba_cb = clf_cb.predict_proba(X_test)

roc_auc = roc_auc_score(y_test, y_pred_proba_cb, multi_class='ovo')
roc_auc

Learning rate set to 0.062221
0:	learn: 0.5980770	test: 0.5976106	best: 0.5976106 (0)	total: 78.8ms	remaining: 1m 18s
50:	learn: 0.2222342	test: 0.2268459	best: 0.2268459 (50)	total: 2.65s	remaining: 49.3s
100:	learn: 0.2114079	test: 0.2226119	best: 0.2226119 (100)	total: 5.21s	remaining: 46.4s
150:	learn: 0.2046259	test: 0.2216911	best: 0.2216900 (149)	total: 7.78s	remaining: 43.8s
200:	learn: 0.1979215	test: 0.2211451	best: 0.2211451 (200)	total: 10.3s	remaining: 40.9s
250:	learn: 0.1920818	test: 0.2210089	best: 0.2209226 (211)	total: 12.8s	remaining: 38.1s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.2209225926
bestIteration = 211

Shrink model to first 212 iterations.


0.8843693782401463

* Сделайте бленд, посчитав среднее между тремя предсказаниями
* Посчитайте roc_auc

In [26]:
voting_predictions = (y_pred_proba_cb + y_pred_proba_xb + y_pred_proba_rf) / 3

roc_auc = roc_auc_score(y_test, voting_predictions, multi_class='ovo')
roc_auc

0.8847122723547554

In [27]:
voting_predictions = y_pred_proba_cb * 0.5 + y_pred_proba_xb * 0.4 + y_pred_proba_rf * 0.1

roc_auc = roc_auc_score(y_test, voting_predictions, multi_class='ovo')
roc_auc

0.8856574629041921

# 4. Бленд с весами. Оптимизация значений весов (10 мин).

* Мы можем добавить веса каждому из пресказаний бленда
* Подберем оптюной значения весов

In [28]:
import optuna

optuna.logging.set_verbosity(optuna.logging.ERROR)
def objective(trial):

    a = trial.suggest_float('a', 0, 1)
    b = trial.suggest_float('b', 0, 1 - a)
    c = 1 - a - b
    
    voting_predictions = a * y_pred_proba_cb + b * y_pred_proba_xb + c * y_pred_proba_rf
    
    roc_auc = roc_auc_score(y_test, voting_predictions, multi_class='ovo')
    
    return roc_auc

In [29]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=1000)

best_weights = study.best_params
best_weights

/opt/conda/lib/python3.10/site-packages/optuna/samplers/_tpe/sampler.py:529: RuntimeWarning: invalid value encountered in subtract
  acq_func_vals = log_likelihoods_below - log_likelihoods_above
/opt/conda/lib/python3.10/site-packages/optuna/samplers/_tpe/sampler.py:529: RuntimeWarning: invalid value encountered in subtract
  acq_func_vals = log_likelihoods_below - log_likelihoods_above
/opt/conda/lib/python3.10/site-packages/optuna/samplers/_tpe/sampler.py:529: RuntimeWarning: invalid value encountered in subtract
  acq_func_vals = log_likelihoods_below - log_likelihoods_above
/opt/conda/lib/python3.10/site-packages/optuna/samplers/_tpe/sampler.py:529: RuntimeWarning: invalid value encountered in subtract
  acq_func_vals = log_likelihoods_below - log_likelihoods_above


{'a': 0.5979961063539885, 'b': 0.27531083356672875}

In [30]:
best_a = best_weights['a']
best_b = best_weights['b']
best_c = 1 - best_a - best_b

best_voting_predictions = best_a * y_pred_proba_cb + best_b * y_pred_proba_xb + best_c * y_pred_proba_rf

best_roc_auc = roc_auc_score(y_test, best_voting_predictions, multi_class='ovo')
best_roc_auc

0.8859283722298007

* Выведите на экран, какие веса мы подобрали

In [31]:
print(best_a, best_b, best_c)

0.5979961063539885 0.27531083356672875 0.12669306007928277


# 5. Бленд с публичным решением.

* Получите предсказания на тестовых данных разными мл моделями. Сделайте бленд.
* Скачайте предсказания из публичнном ноуте. Например [из этого](https://www.kaggle.com/code/khsamaha/spd-multi-label-xgboost-mlr-r/output). Сблендите с вашими предсказаниями. 
* *Отошлите сабмишн на лб.

In [32]:
cb_preds = clf_cb.predict_proba(test.iloc[:,1:])
xgb_preds = clf_xb.predict_proba(test.iloc[:,1:])

predicts = 0.6 * cb_preds + 0.4 * xgb_preds
# best_a, best_b - наши веса

Добавьте предикты из ноута по ссылке.

Для этого нажмите add input и сделайте поиск по названию ноутбука.

Получите предсказания из ноута и сбледите с нашими.

In [33]:
sub2 = pd.read_csv('/kaggle/input/spd-multi-label-xgboost-mlr-r/submission.csv')
sub3 = pd.read_csv('/kaggle/input/ps4e3-steel-plate-fault-prediction-multilabel/submission_comb.csv')

Добавте к нашему бленду еще один или несколько [публичных ноутбуков](https://www.kaggle.com/competitions/playground-series-s4e3/code?competitionId=68699&sortBy=scoreDescending&excludeNonAccessedDatasources=true) с предсказаниями (можно поставить фильтр Public score)

Старайтесь выбирать для бленда модели, которые не обучали сами.

In [34]:
preds_fin = sub2.iloc[:,1:].values * 0.1 + predicts * 0.1 + sub3.iloc[:,1:].values * 0.8

In [35]:
submission = pd.DataFrame(
    preds_fin,
    columns=targets,
    index=test.id
).reset_index()
submission.to_csv('submission.csv', index=False)
submission.head(5)

,id,Pastry,Z_Scratch,K_Scatch,Stains,Dirtiness,Bumps,Other_Faults
0,19219,0.482327,0.003502,0.004232,0.007867,0.020232,0.155492,0.326437
1,19220,0.287892,0.025048,0.011333,0.009184,0.148615,0.164321,0.345433
2,19221,0.005542,0.039518,0.041839,0.009984,0.011320,0.327451,0.534944
3,19222,0.114475,0.006026,0.002099,0.010245,0.014127,0.423671,0.417156
4,19223,0.006622,0.005256,0.002399,0.011700,0.009643,0.628359,0.335519
